In [1]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from textblob import TextBlob
# import seaborn as sn

In [3]:
#Libraries 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import os

# Import functions for data preprocessing & data preparation
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string
from string import punctuation
import nltk
import re

In [9]:
data = pd.read_csv('C:/Users/rajat.n_elxer/Downloads/amazon_reviews.csv')
data.columns
data1=data.drop(['Unnamed: 0','reviewerName','overall','reviewTime','day_diff','helpful_yes','helpful_no','total_vote','score_pos_neg_diff','score_average_rating','wilson_lower_bound'],axis=1)
data1

,reviewText
0,No issues.
1,"Purchased this for my device, it worked as adv..."
2,it works as expected. I should have sprung for...
3,This think has worked out great.Had a diff. br...
4,"Bought it with Retail Packaging, arrived legit..."
...,...
4910,I bought this Sandisk 16GB Class 10 to use wit...
4911,Used this for extending the capabilities of my...
4912,Great card that is very fast and reliable. It ...
4913,Good amount of space for the stuff I want to d...


In [15]:
data1.reviewText[5]

"It's mini storage.  It doesn't do anything else and it's not supposed to.  I purchased it to add additional storage to my Microsoft Surface Pro tablet which only come in 64 and 128 GB.  It does what it's supposed to and SanDisk has a long standing reputation that speaks for itself."

Using vader_lexicon

In [ ]:
nltk.download('vader_lexicon')


data1["reviewText"] = data1["reviewText"].fillna("")  

sentiments = SentimentIntensityAnalyzer()

data1["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data1["reviewText"]]
data1["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data1["reviewText"]]
data1["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data1["reviewText"]]
data1['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in data1["reviewText"]]

score = data1["Compound"].values
sentiment = []

for i in score:
    if i >= 0.05:
        sentiment.append('Positive')
    elif i <= -0.05:
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')

data1["Sentiment"] = sentiment
data1.head(11)

Using TextBlob

In [ ]:
lab=data1['reviewText'].values
n_review=[]
for i in lab:
    tha=(TextBlob(i)).sentiment[0]
    if tha>0.2:
        j='Pos'
        n_review.append(j)
    elif tha<-0.2:
        t='Neg'
        n_review.append(t)
    else:
        r='Neutral'
        n_review.append(r)
print(n_review)
data1["textblob_sentiements"]=n_review

Using sentiwordnet

In [ ]:
from nltk.corpus import sentiwordnet
nltk.download('sentiwordnet')


data1["reviewText"] = data1["reviewText"].fillna("")  

sentiments = SentimentIntensityAnalyzer()

data1["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data1["reviewText"]]
data1["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data1["reviewText"]]
data1["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data1["reviewText"]]
data1['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in data1["reviewText"]]

score = data1["Compound"].values
sentiment = []

for i in score:
    if i >= 0.05:
        sentiment.append('Positive')
    elif i <= -0.05:
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')

data1["Sentiment SentiWordNet"] = sentiment
data1.head(11)

Using Afinn

In [ ]:
from afinn import Afinn
import pandas as pd

# Assuming data1 is your DataFrame
data1["reviewText"] = data1["reviewText"].fillna("")

# Initialize AFINN
afinn = Afinn()

# Calculate sentiment scores
data1["Sentiment_Score"] = data1["reviewText"].apply(lambda x: afinn.score(x))

# Categorize sentiments based on scores
data1["Sentiment Afinn"] = data1["Sentiment_Score"].apply(lambda x: 'Positive' if x > 2 else 'Negative' if x < -2 else 'Neutral')


data2=data1.drop(['Positive','Negative','Neutral','Compound'],axis=1)
data2.head(51)


In [42]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer() 
snowball_stemer = SnowballStemmer(language="english")
lzr = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rajat.n_elxer\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [ ]:
def text_processing(text):   
    # convert text into lowercase
    text = text.lower()

    # remove new line characters in text
    text = re.sub(r'\n',' ', text)
    
    # remove punctuations from text
    text = re.sub('[%s]' % re.escape(punctuation), "", text)
    
    # remove references and hashtags from text
    text = re.sub("^a-zA-Z0-9$,.", "", text)
    
    # remove multiple spaces from text
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # remove special characters from text
    text = re.sub(r'\W', ' ', text)

    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])
    
    # stemming using porter stemmer from nltk package - msh a7sn 7aga - momken: lancaster, snowball
    # text=' '.join([porter_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([lancaster_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([snowball_stemer.stem(word) for word in word_tokenize(text)])
    
    # lemmatizer using WordNetLemmatizer from nltk package
    text=' '.join([lzr.lemmatize(word) for word in word_tokenize(text)])

    return text

In [ ]:
nltk.download('omw-1.4')
data_copy = data2.copy()
data_copy.Comment = data_copy.Comment.apply(lambda text: text_processing(text))

In [ ]:
le = LabelEncoder()
data_copy['Sentiment'] = le.fit_transform(data_copy['Sentiment'])

In [ ]:
processed_data = {
    'Sentence':data_copy.Comment,
    'Sentiment':data_copy['Sentiment']
}

processed_data = pd.DataFrame(processed_data)
processed_data.head()

In [ ]:
processed_data['Sentiment'].value_counts()

In [ ]:
df_neutral = processed_data[(processed_data['Sentiment']==1)] 
df_negative = processed_data[(processed_data['Sentiment']==0)]
df_positive = processed_data[(processed_data['Sentiment']==2)]

# upsample minority classes
df_negative_upsampled = resample(df_negative, 
                                 replace=True,    
                                 n_samples= 205, 
                                 random_state=42)  

df_neutral_upsampled = resample(df_neutral, 
                                 replace=True,    
                                 n_samples= 205, 
                                 random_state=42)  


# Concatenate the upsampled dataframes with the neutral dataframe
final_data = pd.concat([df_negative_upsampled,df_neutral_upsampled,df_positive])

In [ ]:
final_data['Sentiment'].value_counts()

In [ ]:
corpus = []
for sentence in final_data['Sentence']:
    corpus.append(sentence)
corpus[0:5]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
y = final_data.iloc[:, -1].values

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
classifier = GaussianNB()
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
nb_score = accuracy_score(y_test, y_pred)
print('accuracy',nb_score)